# Récupération de données

In [33]:
pip install xgboost

     ---------------------------------------- 70.9/70.9 MB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
import numpy as np
import pandas as pd
import xgboost as xgb
import sqlite3
import streamlit as st
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import metrics

In [43]:
import pandas as pd
import requests

api_key = "f73c8544980c792c998a710ef5fcc956"
lat = 14.6928  # Latitude de Dakar
lon = -17.4467  # Longitude de Dakar

url = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={api_key}"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()

    # Créer une liste pour stocker les données de chaque période
    forecast_data = []
    
    # Parcourir les prévisions pour chaque période
    for forecast in data["list"]:
        forecast_data.append({
            "Date": forecast["dt_txt"],
            "Temperature": forecast["main"]["temp"],
            "Humidity": forecast["main"]["humidity"],
            "Pressure": forecast["main"]["pressure"],
            "Wind Speed": forecast["wind"]["speed"],
            "Wind Direction": forecast["wind"]["deg"],
            "Description": forecast["weather"][0]["description"]
        })

    # Créer un DataFrame avec les données
    df = pd.DataFrame(forecast_data)
    
    # Afficher le DataFrame
    print(df)
    
    
else:
    print("La requête a échoué avec le code d'erreur :", response.status_code)


                   Date  Temperature  Humidity  Pressure  Wind Speed  \
0   2023-07-18 21:00:00       301.65        77      1013        3.04   
1   2023-07-19 00:00:00       300.82        80      1014        3.05   
2   2023-07-19 03:00:00       300.29        82      1015        2.77   
3   2023-07-19 06:00:00       300.25        82      1015        2.67   
4   2023-07-19 09:00:00       300.42        81      1015        2.14   
5   2023-07-19 12:00:00       300.92        79      1016        1.63   
6   2023-07-19 15:00:00       301.08        80      1014        3.60   
7   2023-07-19 18:00:00       301.02        81      1013        3.22   
8   2023-07-19 21:00:00       300.78        82      1014        3.32   
9   2023-07-20 00:00:00       300.80        82      1015        3.34   
10  2023-07-20 03:00:00       300.68        82      1014        2.95   
11  2023-07-20 06:00:00       300.48        82      1014        2.12   
12  2023-07-20 09:00:00       300.60        82      1015        

In [44]:
df.isnull().sum()

Date              0
Temperature       0
Humidity          0
Pressure          0
Wind Speed        0
Wind Direction    0
Description       0
dtype: int64

In [45]:
df.describe()

,Temperature,Humidity,Pressure,Wind Speed,Wind Direction
count,40.000000,40.000000,40.000000,40.000000,40.000000
mean,300.534250,76.850000,1013.525000,4.791500,277.600000
std,0.476805,4.704335,1.320208,2.006741,34.595687
min,299.670000,67.000000,1011.000000,1.630000,205.000000
25%,300.137500,72.750000,1013.000000,3.047500,255.750000
50%,300.540000,79.000000,1013.500000,4.735000,276.500000
75%,300.805000,81.000000,1014.250000,6.212500,292.250000
max,301.650000,82.000000,1016.000000,9.630000,357.000000


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            40 non-null     object 
 1   Temperature     40 non-null     float64
 2   Humidity        40 non-null     int64  
 3   Pressure        40 non-null     int64  
 4   Wind Speed      40 non-null     float64
 5   Wind Direction  40 non-null     int64  
 6   Description     40 non-null     object 
dtypes: float64(2), int64(3), object(2)
memory usage: 2.3+ KB


In [47]:
df.corr()

C:\Users\Hp\AppData\Local\Temp\ipykernel_19076\1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,Temperature,Humidity,Pressure,Wind Speed,Wind Direction
Temperature,1.000000,0.483493,0.408588,-0.165220,0.037195
Humidity,0.483493,1.000000,0.508428,-0.158840,0.600040
Pressure,0.408588,0.508428,1.000000,-0.529421,0.209065
Wind Speed,-0.165220,-0.158840,-0.529421,1.000000,-0.082176
Wind Direction,0.037195,0.600040,0.209065,-0.082176,1.000000


In [65]:
df

,Date,Temperature,Humidity,Pressure,Wind Speed,Wind Direction,Description,Year,Month,Day,Hour,Description_Code
0,2023-07-18,301.65,77,1013,3.04,255,broken clouds,2023,7,18,21,0
1,2023-07-19,300.82,80,1014,3.05,233,light rain,2023,7,19,0,2
2,2023-07-19,300.29,82,1015,2.77,276,light rain,2023,7,19,3,2
3,2023-07-19,300.25,82,1015,2.67,346,light rain,2023,7,19,6,2
4,2023-07-19,300.42,81,1015,2.14,354,light rain,2023,7,19,9,2
5,2023-07-19,300.92,79,1016,1.63,318,overcast clouds,2023,7,19,12,4
6,2023-07-19,301.08,80,1014,3.60,357,light rain,2023,7,19,15,2
7,2023-07-19,301.02,81,1013,3.22,337,overcast clouds,2023,7,19,18,4
8,2023-07-19,300.78,82,1014,3.32,291,overcast clouds,2023,7,19,21,4
9,2023-07-20,300.80,82,1015,3.34,263,overcast clouds,2023,7,20,0,4


In [64]:
df.to_excel("output.xlsx",sheet_name='Sheet_name_1') 

In [83]:
X=df.drop(['Date','Temperature','Description'], axis=1)

In [84]:
y=df['Temperature']

In [80]:
from sklearn import preprocessing

transformer = preprocessing.MinMaxScaler().fit(X[['Pressure','Humidity','Wind Speed','Wind Direction']])
X_transformed = transformer.transform(X[['Pressure','Humidity','Wind Speed','Wind Direction']])

In [81]:
print(f'Min : ',round(X_transformed.min(),2),', Max :', round(X_transformed.max(),2))
print(f'Mean : ',round(X_transformed.mean(),2),', Standard Deviation :', round(X_transformed.std(),2))

Min :  0.0 , Max : 1.0
Mean :  0.51 , Standard Deviation : 0.28


In [85]:
# Division des données en ensembles d'apprentissage et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [86]:
# Création du modèle XGBoost
model = xgb.XGBRegressor()


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Date              40 non-null     object  
 1   Temperature       40 non-null     float64 
 2   Humidity          40 non-null     int64   
 3   Pressure          40 non-null     int64   
 4   Wind Speed        40 non-null     float64 
 5   Wind Direction    40 non-null     int64   
 6   Description       40 non-null     category
 7   Year              40 non-null     int64   
 8   Month             40 non-null     int64   
 9   Day               40 non-null     int64   
 10  Hour              40 non-null     int64   
 11  Description_Code  40 non-null     int8    
dtypes: category(1), float64(2), int64(7), int8(1), object(1)
memory usage: 3.5+ KB


In [87]:
# Entraînement du modèle
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [88]:
# Prévisions sur les données de test
y_pred = model.predict(X_test)

In [89]:
# Évaluation des performances en utilisant l'erreur quadratique moyenne (RMSE)

print("MSE", mean_squared_error(y_test,y_pred))
print("R squared", metrics.r2_score(y_test,y_pred))

MSE 0.05110260301009066
R squared 0.8356226614139133


In [90]:
print("Accuracy={:.2f}".format(model.score(X_test, y_test)))

Accuracy=0.84


On a une précision de 84% ce qui veut dire 84% du temps notre modèle effectue une bonne prédiction,il arrive à prédire la temperature si on a la pression,l'humidité,la vitesse et le direction du vent. 

In [91]:
#Créez un chatbot en utilisant Python et intégrez la reconnaissance vocale


In [94]:
import streamlit as st
import speech_recognition as sr

In [95]:
def transcribe_speech():
    # Initialize recognizer class ##Initialisation de la classe de reconnaissance
    r = sr.Recognizer()
    # Reading Microphone as source ##Lecture Microphone comme source
    with sr.Microphone() as source:
        st.info("Speak now...")
        # listen for speech and store in audio_text variable ##écouter la parole et la stocker dans la variable audio_text
        audio_text = r.listen(source)
        st.info("Transcribing...")

        try:
            # using Google Speech Recognition ###en utilisant la reconnaissance vocale de Google
            text = r.recognize_google(audio_text)
            return text
        except:
            return "Sorry, I did not get that."

In [96]:
def main():
    st.title("Chatbot avec reconnaissance vocale")
    user_input = st.text_input("Vous: ", "")
    if st.button("Envoyer"):
        response = chatbot.get_response(user_input)
        st.text_area("Chatbot: ", value=str(response))
    st.subheader("Reconnaissance vocale")
    st.write("Cliquez sur le bouton ci-dessous et parlez pour avoir une réponse du chatbot :")
    if st.button("Reconnaissance vocale"):
        st.write("Dites quelque chose...")
        voice_input =transcribe_speech()
        if voice_input:
            st.text_area("Vous: ", value=voice_input)
            response = chatbot.get_response(voice_input)
            st.text_area("Chatbot: ", value=str(response))
        else:
            st.write("Aucune entrée vocale détectée.")


In [97]:
if __name__ == '__main__':
    main()


2023-07-18 21:42:46.980 
  command:

    streamlit run C:\Users\Hp\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
